In [1]:
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SC
from openpyxl import load_workbook
from openpyxl.utils.cell import get_column_letter

In [2]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]
credentials = SC.from_json_keyfile_name('json_key_file_path', scope) # json_key_file_path => service account key file path
gs = gspread.authorize(credentials)

In [3]:
spread = gs.create('New Document')
spread.share('john@email.com', perm_type='user', role='reader') # john@email.com => email address you'd like to share with

<Response [200]>

In [4]:
data_path = 'C:\\Users\\username\\folder_path'
file_list = os.listdir(data_path)
for file in file_list:
    if file[-4:] != 'xlsx':
        continue
    file_path = os.path.join(data_path, file)
    wb = load_workbook(file_path, read_only=True)
    ws = wb.active
    row_cnt = ws.max_row
    col_cnt = ws.max_column
    worksheet = spread.add_worksheet(file, row_cnt, col_cnt)
    cells = worksheet.range('A1:'+get_column_letter(col_cnt)+str(row_cnt))
    idx = 0
    for row in ws.iter_rows():
        for cell in row:
            cells[idx].value = str(cell.value)
            idx += 1
    worksheet.update_cells(cells)